Backend code for the Movie Booking System

Step 1: Install Required Dependencies

Ensure you have the necessary dependencies installed, including Flask, pymongo, and requests for making HTTP requests:

In [ ]:
!pip install Flask pymongo requests pyspark

In [1]:
from flask import Flask, jsonify, request
import threading
import requests
import time
from datetime import datetime
from pyspark.sql import SparkSession
import random
import pymongo
from pymongo import ReturnDocument
from pymongo import MongoClient
import os

In [2]:
app = Flask(__name__)

In [3]:
# Sample theater data
theaters_data = [
    {'area': 'Downtown', 'pincode': 64106, 'theatre': 'AMC Mainstreet 6'},
    {'area': 'Westport', 'pincode': 64111, 'theatre': 'Alamo Drafthouse Cinema'},
    {'area': 'Crown Center', 'pincode': 64108, 'theatre': 'Union Station Extreme Screen'},
    {'area': 'Country Club Plaza', 'pincode': 64112, 'theatre': 'AMC Ward Parkway 14'},
    {'area': 'Power & Light District', 'pincode': 64105, 'theatre': 'B&B Theatres Mainstreet KC'},
    {'area': 'North Kansas City', 'pincode': 64116, 'theatre': 'Screenland Armour'},
    {'area': 'Overland Park', 'pincode': 66223, 'theatre': 'Cinetopia Overland Park 18'},
    {'area': 'Overland Park', 'pincode': 66204, 'theatre': 'Rio Theatre'},
    {'area': 'Kansas City Downtown', 'pincode': 64106, 'theatre': 'Regal Kansas City'}
]

# Sample movie data
movies_data = [
    {'title': 'The Shawshank Redemption', 'genre': 'Drama', 'language': 'English', 'theatre': 'AMC Mainstreet 6'},
    {'title': 'The Godfather', 'genre': 'Crime', 'language': 'English', 'theatre': 'AMC Mainstreet 6'},
    {'title': 'The Dark Knight', 'genre': 'Action', 'language': 'English', 'theatre': 'Alamo Drafthouse Cinema'},
    {'title': 'Pulp Fiction', 'genre': 'Crime', 'language': 'English', 'theatre': 'Union Station Extreme Screen'},
    {'title': 'Inception', 'genre': 'Sci-Fi', 'language': 'English', 'theatre': 'B&B Theatres Mainstreet KC'},
    {'title': 'Avengers: Endgame', 'genre': 'Action', 'language': 'English', 'theatre': 'Regal Kansas City'},
    {'title': 'The Matrix', 'genre': 'Sci-Fi', 'language': 'English', 'theatre': 'Cinetopia Overland Park 18'}
]

# Sample seat availability for each movie at each theater
seat_availability = {
    ('AMC Mainstreet 6', 'The Shawshank Redemption'): 100,
    ('AMC Mainstreet 6', 'The Godfather'): 50,
    ('Alamo Drafthouse Cinema', 'The Dark Knight'): 75,
    ('Alamo Drafthouse Cinema', 'Pulp Fiction'): 35,
    ('Rio Theatre', 'The Matrix'): 35,
    ('Regal Union Square', 'The Matrix'): 100,
    ('Cinemark Tinseltown', 'Inception'): 10,
}

# Sample in-memory dataset for theatres with base prices using Dask
theatre_data = [
    {"theatre": "AMC Mainstreet 6", "base_price": 12},
    {"theatre": "Regal Union Square", "base_price": 15},
    {"theatre": "Cinemark Tinseltown", "base_price": 10},
    {"theatre": "Alamo Drafthouse Cinema", "base_price": 15},
    {"theatre": "Rio Theatre", "base_price": 25},
]

In [4]:
from pymongo import MongoClient

# MongoDB connection
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017/")
client = MongoClient(MONGO_URI, maxPoolSize=100)
db = client['movie_booking_system8']

# Collections for different entities
theatre_collection = db['theatres']
movie_collection = db['movies']
seat_collection = db['seats']
user_collection = db['users']
admin_collection = db['admins']
booking_collection = db['bookings']  # Add bookings collection


# Updated Sample Data for Users
users_data = [
    {"user_id": 1, "name": "Alice Johnson", "email": "alice@example.com", "mobile_number": "123-456-7890", 
     "username": "alicejohnson", "password": "alicepass"},
    {"user_id": 2, "name": "Bob Smith", "email": "bob@example.com", "mobile_number": "234-567-8901", 
     "username": "bobsmith", "password": "bobpass"},
    {"user_id": 3, "name": "Carol White", "email": "carol@example.com", "mobile_number": "345-678-9012", 
     "username": "carolwhite", "password": "carolpass"},
    {"user_id": 4, "name": "David Brown", "email": "david@example.com", "mobile_number": "456-789-0123", 
     "username": "davidbrown", "password": "davidpass"},
    {"user_id": 5, "name": "Emma Davis", "email": "emma@example.com", "mobile_number": "567-890-1234", 
     "username": "emmadavis", "password": "emmapass"},
    {"user_id": 6, "name": "Frank Thomas", "email": "frank@example.com", "mobile_number": "678-901-2345", 
     "username": "frankthomas", "password": "frankpass"},
    {"user_id": 7, "name": "Grace Lee", "email": "grace@example.com", "mobile_number": "789-012-3456", 
     "username": "gracelee", "password": "gracepass"},
    {"user_id": 8, "name": "Hank Taylor", "email": "hank@example.com", "mobile_number": "890-123-4567", 
     "username": "hanktaylor", "password": "hankpass"},
    {"user_id": 9, "name": "Ivy Scott", "email": "ivy@example.com", "mobile_number": "901-234-5678", 
     "username": "ivyscott", "password": "ivypass"},
    {"user_id": 10, "name": "Jack Turner", "email": "jack@example.com", "mobile_number": "012-345-6789", 
     "username": "jackturner", "password": "jackpass"}
]

# Updated Sample Data for Admins
admins_data = [
    {"admin_id": 1, "name": "Admin One", "email": "admin1@example.com", "mobile_number": "987-654-3210", 
     "username": "admin1", "password": "adminpass1", "theatre": "AMC Mainstreet 6"},
    {"admin_id": 2, "name": "Admin Two", "email": "admin2@example.com", "mobile_number": "876-543-2109", 
     "username": "admin2", "password": "adminpass2", "theatre": "Regal Kansas City"},
    {"admin_id": 3, "name": "Admin Three", "email": "admin3@example.com", "mobile_number": "765-432-1098", 
     "username": "admin3", "password": "adminpass3", "theatre": "Alamo Drafthouse Cinema"},
    {"admin_id": 4, "name": "Admin Four", "email": "admin4@example.com", "mobile_number": "654-321-0987", 
     "username": "admin4", "password": "adminpass4", "theatre": "B&B Theatres Mainstreet KC"},
    {"admin_id": 5, "name": "Admin Five", "email": "admin5@example.com", "mobile_number": "543-210-9876", 
     "username": "admin5", "password": "adminpass5", "theatre": "Union Station Extreme Screen"},
    {"admin_id": 6, "name": "Admin Six", "email": "admin6@example.com", "mobile_number": "432-109-8765", 
     "username": "admin6", "password": "adminpass6", "theatre": "Cinetopia Overland Park 18"},
    {"admin_id": 7, "name": "Admin Seven", "email": "admin7@example.com", "mobile_number": "321-098-7654", 
     "username": "admin7", "password": "adminpass7", "theatre": "Rio Theatre"},
    {"admin_id": 8, "name": "Admin Eight", "email": "admin8@example.com", "mobile_number": "210-987-6543", 
     "username": "admin8", "password": "adminpass8", "theatre": "Screenland Armour"},
    {"admin_id": 9, "name": "Admin Nine", "email": "admin9@example.com", "mobile_number": "109-876-5432", 
     "username": "admin9", "password": "adminpass9", "theatre": "AMC Ward Parkway 14"},
    {"admin_id": 10, "name": "Admin Ten", "email": "admin10@example.com", "mobile_number": "098-765-4321", 
     "username": "admin10", "password": "adminpass10", "theatre": "Power & Light District Theatre"}
]

# Inserting sample data into MongoDB collections
user_collection.insert_many(users_data)
admin_collection.insert_many(admins_data)
theatre_collection.insert_many(theaters_data)
movie_collection.insert_many(movies_data)
seat_collection.insert_many(theatre_data)

InsertManyResult([ObjectId('67285cc5a08e0af058903fbd'), ObjectId('67285cc5a08e0af058903fbe'), ObjectId('67285cc5a08e0af058903fbf'), ObjectId('67285cc5a08e0af058903fc0'), ObjectId('67285cc5a08e0af058903fc1')], acknowledged=True)

In [5]:
import dask.dataframe as dd
import pandas as pd
# Create a Dask DataFrame for theatres
theatre_df = dd.from_pandas(pd.DataFrame(theatre_data), npartitions=1)

# List to store booking records as a Dask DataFrame
bookings_data = {
    "theatre": [],
    "base_price": [],
    "seats_sold": [],
    "date": []
}
bookings_dask_df = dd.from_pandas(pd.DataFrame(bookings_data), npartitions=1)
booking_collection = db['bookings']  # Replace with your bookings collection name

# Simulate festival season multiplier (hardcoded)
festival_seasons = {
    "Diwali": "11-14",
    "Christmas": "12-25",
    "New Year": "01-01"
}

# Function to calculate dynamic price based on the date and number of seats
def get_dynamic_price(base_price, seats, booking_date):
    multiplier = 1.0
    booking_month_day = booking_date.strftime("%m-%d")
    for season, date in festival_seasons.items():
        if booking_month_day == date:
            multiplier = 1.5  # Increase price by 50% during festivals
    return base_price * multiplier * seats

In [6]:
@app.route('/user/register', methods=['POST'])
def register_user():
    user_data = request.json
    user_collection.insert_one(user_data)
    return jsonify({'message': 'User registered successfully'}), 201

@app.route('/user/login', methods=['POST'])
def login_user():
    login_data = request.json
    user = user_collection.find_one({'username': login_data['username']})
    
    if user and user['password'] == login_data['password']:
        return jsonify({"message": "Logged in successfully"}), 200
    return jsonify({"error": "Invalid username or password"}), 401

@app.route('/admin/register', methods=['POST'])
def register_admin():
    admin_data = request.json
    admin_collection.insert_one(admin_data)
    return jsonify({'message': 'Admin registered successfully'}), 201

@app.route('/admin/login', methods=['POST'])
def login_admin():
    login_data = request.json
    admin = admin_collection.find_one({'username': login_data['username']})
    
    if admin and admin['password'] == login_data['password']:
        return jsonify({"message": "Logged in successfully"}), 200
    return jsonify({"error": "Invalid username or password"}), 401

In [7]:
# Error handler for undefined routes
@app.errorhandler(404)
def not_found(error):
    return jsonify({"error": "Route not found"}), 404

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "running"}), 200

In [8]:
@app.route('/theater/add', methods=['POST'])
def add_theater():
    theater_data = request.json
    theater_id = theatre_collection.insert_one(theater_data).inserted_id
    return jsonify({'message': 'Theater added successfully', 'theater_id': str(theater_id)}), 201

@app.route('/theater/update/<theater_id>', methods=['PUT'])
def update_theater(theater_id):
    theater_data = request.json
    result = theatre_collection.update_one({'_id': ObjectId(theater_id)}, {'$set': theater_data})
    if result.matched_count > 0:
        return jsonify({'message': 'Theater updated successfully'}), 200
    else:
        return jsonify({'message': 'Theater not found'}), 404

@app.route('/availability/<theatre>/<movie>', methods=['GET'])
def check_availability(theatre, movie):
    available_seats = seat_availability.get((theatre, movie), 0)
    return jsonify({"available_seats": available_seats})

total_bookings = {theatre["theatre"]: 0 for theatre in theatre_data}
total_revenue = {theatre["theatre"]: 0 for theatre in theatre_data}

def get_base_price(theatre):
    for data in theatre_data:
        if data["theatre"] == theatre:
            return data["base_price"]
    return None

@app.route('/pricing/<theatre>', methods=['GET'])
def get_pricing(theatre):
    base_price = get_base_price(theatre)
    if base_price is not None:
        return jsonify({"base_price": base_price})
    return jsonify({"error": "Theatre not found"}), 404

@app.route('/book', methods=['POST'])
def book_tickets():
    data = request.json
    theatre = data.get("theatre")
    movie = data.get("movie")
    seats = data.get("seats")

    # Check seat availability
    available_seats = seat_availability.get((theatre, movie), 0)
    if seats > available_seats:
        return jsonify({"msg": f"Sorry, only {available_seats} seats are available for {movie} at {theatre}."}), 400
    
    # Get base price for the theatre
    base_price = get_base_price(theatre)
    if base_price is None:
        return jsonify({"error": "Theatre not found"}), 404
    
    # Calculate total price
    total_price = base_price * seats
    
    # Create booking data
    booking_data = {
        "theatre": theatre,
        "movie": movie,
        "seats": seats,
        "total_price": total_price
    }
    
    # Update seat availability and bookings
    seat_availability[(theatre, movie)] -= seats
    total_bookings[theatre] += seats
    total_revenue[theatre] += total_price
    
    return jsonify(booking_data), 200
    
@app.route('/bookings', methods=['GET'])
def get_bookings():
    result = []
    for theatre in theatre_data:
        theatre_name = theatre["theatre"]
        booked_seats = total_bookings[theatre_name]
        revenue = total_revenue[theatre_name]
        remaining_seats = {}
        for movie in seat_availability:
            if movie[0] == theatre_name:
                remaining_seats[movie[1]] = seat_availability[movie]
        result.append({
            "theatre": theatre_name,
            "booked_seats": booked_seats,
            "revenue": revenue,
            "remaining_seats": remaining_seats
        })
    return jsonify(result)

@app.route('/users', methods=['GET'])
def get_users():
    users = list(user_collection.find({}))
    for user in users:
        user['_id'] = str(user['_id'])
    return jsonify(users), 200

@app.route('/theaters', methods=['GET'])
def get_theaters():
    theaters = list(theatre_collection.find({}))
    for theater in theaters:
        theater['_id'] = str(theater['_id'])
    return jsonify(theaters), 200

@app.route('/admins', methods=['GET'])
def get_admins():
    admins = list(admin_collection.find({}))
    for admin in admins:
        admin['_id'] = str(admin['_id'])
    return jsonify(admins), 200

@app.route('/movies', methods=['GET'])
def get_movies():
    movies = list(movie_collection.find({}))
    for movie in movies:
        movie['_id'] = str(movie['_id'])
    return jsonify(movies), 200

@app.route('/seats', methods=['GET'])
def get_seats():
    seats = list(seat_collection.find({}))
    for seat in seats:
        seat['_id'] = str(seat['_id'])
    return jsonify(seats), 200

In [9]:
@app.route('/admin', methods=['GET'])
def get_admin_by_id():
    admin_id = request.args.get('admin_id')
    admin = admin_collection.find_one({'admin_id': int(admin_id)})
    if admin:
        admin['_id'] = str(admin['_id'])
        return jsonify(admin), 200
    else:
        return jsonify({"error": "Admin not found"}), 404

@app.route('/movie', methods=['GET'])
def get_movie_by_id():
    movie_id = request.args.get('movie_id')
    movie = movie_collection.find_one({'movie_id': int(movie_id)})
    if movie:
        movie['_id'] = str(movie['_id'])
        return jsonify(movie), 200
    else:
        return jsonify({"error": "Movie not found"}), 404

# API to get user details by username
@app.route('/users/<username>', methods=['GET'])
def get_user_details(username):
    user = user_collection.find_one({"username": username})
    if user:
        user['_id'] = str(user['_id'])  # Convert ObjectId to string
        return jsonify(user), 200
    return jsonify({"message": "User not found"}), 404

@app.route('/bookings/<username>', methods=['GET'])
def get_booking_details(username):
    # Fetch bookings for the specific user from the database
    bookings = list(booking_collection.find({"username": username}))
    
    # Prepare the result
    result = []
    
    # If no bookings found, return an empty list
    if not bookings:
        return jsonify(result), 200
    
    for booking in bookings:
        booking_detail = {
            "movie": booking.get("movie"),  # Assuming "film_name" is the field name
            "theatre": booking.get("theatre"),
            "no_of_seats": booking.get("seats"),
            "total_price": booking.get("total_price")  # Assuming you have this field
        }
        result.append(booking_detail)

    return jsonify(result), 200


In [10]:
def run_app():
    app.run(debug=True, host='0.0.0.0', port=5000, use_reloader=False)

In [11]:
thread = threading.Thread(target=run_app)
thread.start()

# Wait a moment for the server to start
import time
time.sleep(2)

try:
    response = requests.get('http://localhost:5000/health')
    if response.status_code == 200:
        print("Flask app is running:", response.json())
    else:
        print("Failed to start the Flask app")
except Exception as e:
    print(f"Error: {e}")

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.164.226:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Nov/2024 11:04:26] "GET /health HTTP/1.1" 200 -


Flask app is running: {'status': 'running'}


In [12]:
# Function to fetch and display movies
def display_movies():
    # Retrieve all movies from the database
    movies = list(movie_collection.find({}))
    
    # Convert ObjectId to string and create a DataFrame
    for movie in movies:
        movie['_id'] = str(movie['_id'])  # Convert ObjectId to string

    # Create a DataFrame from the list of movies
    df_movies = pd.DataFrame(movies)
    
    # Display the DataFrame
    return df_movies

# Display movies in the notebook
movies_df = display_movies()
movies_df  # This will display the DataFrame in the notebook

,_id,title,genre,language,theatre
0,6727cf8de5a70953d4643e3a,The Shawshank Redemption,Drama,English,AMC Mainstreet 6
1,6727cf8de5a70953d4643e3b,The Godfather,Crime,English,AMC Mainstreet 6
2,6727cf8de5a70953d4643e3c,The Dark Knight,Action,English,Alamo Drafthouse Cinema
3,6727cf8de5a70953d4643e3d,Pulp Fiction,Crime,English,Union Station Extreme Screen
4,6727cf8de5a70953d4643e3e,Inception,Sci-Fi,English,B&B Theatres Mainstreet KC
5,6727cf8de5a70953d4643e3f,Avengers: Endgame,Action,English,Regal Kansas City
6,6727cf8de5a70953d4643e40,The Matrix,Sci-Fi,English,Cinetopia Overland Park 18
7,6727e63b6011035c31964dc5,The Shawshank Redemption,Drama,English,AMC Mainstreet 6
8,6727e63b6011035c31964dc6,The Godfather,Crime,English,AMC Mainstreet 6
9,6727e63b6011035c31964dc7,The Dark Knight,Action,English,Alamo Drafthouse Cinema


In [13]:
# Function to fetch and display users
def display_users(limit=5):
    users = list(user_collection.find({}).limit(limit))
    for user in users:
        user['_id'] = str(user['_id'])  # Convert ObjectId to string
    return pd.DataFrame(users)

# Function to fetch and display admins
def display_admins(limit=5):
    admins = list(admin_collection.find({}).limit(limit))
    for admin in admins:
        admin['_id'] = str(admin['_id'])  # Convert ObjectId to string
    return pd.DataFrame(admins)

# Function to fetch and display seat availability
def display_seat_availability():
    seats = list(seat_collection.find({}))
    for seat in seats:
        seat['_id'] = str(seat['_id'])  # Convert ObjectId to string
    return pd.DataFrame(seats)

# Display users, admins, and seat availability in the notebook
users_df = display_users()
admins_df = display_admins()
seats_df = display_seat_availability()

# Display the DataFrames
users_df, admins_df, seats_df  # This will display the DataFrames in the notebook

(                        _id  user_id           name              email  \
 0  6727cf8de5a70953d4643e1d        1  Alice Johnson  alice@example.com   
 1  6727cf8de5a70953d4643e1e        2      Bob Smith    bob@example.com   
 2  6727cf8de5a70953d4643e1f        3    Carol White  carol@example.com   
 3  6727cf8de5a70953d4643e20        4    David Brown  david@example.com   
 4  6727cf8de5a70953d4643e21        5     Emma Davis   emma@example.com   
 
   mobile_number      username   password  
 0  123-456-7890  alicejohnson  alicepass  
 1  234-567-8901      bobsmith    bobpass  
 2  345-678-9012    carolwhite  carolpass  
 3  456-789-0123    davidbrown  davidpass  
 4  567-890-1234     emmadavis   emmapass  ,
                         _id  admin_id         name               email  \
 0  6727cf8de5a70953d4643e27         1    Admin One  admin1@example.com   
 1  6727cf8de5a70953d4643e28         2    Admin Two  admin2@example.com   
 2  6727cf8de5a70953d4643e29         3  Admin Three  admin

In [14]:
#To verify user/login api
import requests

# Set up the login data
login_data = {
    "username": "alicejohnson",  # Replace with a valid username from your database
    "password": "alicepass"      # Replace with the corresponding password
}

# Send the login request
response = requests.post('http://localhost:5000/user/login', json=login_data)

# Print the response
print(f"Status Code: {response.status_code}")
print("Response JSON: ", response.json())

127.0.0.1 - - [04/Nov/2024 11:04:39] "POST /user/login HTTP/1.1" 200 -


Status Code: 200
Response JSON:  {'message': 'Logged in successfully'}


In [15]:
import requests

# Set up the booking data
booking_data = {
    "theatre": "AMC Mainstreet 6",  # Replace with a valid theatre name from your dataset
    "movie": "The Shawshank Redemption",  # Replace with a valid movie title from your dataset
    "seats": 6  # Replace with the number of seats you want to book
}

# Send the booking request
response = requests.post('http://localhost:5000/book', json=booking_data)

# Print the response
print(f"Status Code: {response.status_code}")
print("Response JSON: ", response.json())


127.0.0.1 - - [04/Nov/2024 11:04:42] "POST /book HTTP/1.1" 200 -


Status Code: 200
Response JSON:  {'movie': 'The Shawshank Redemption', 'seats': 6, 'theatre': 'AMC Mainstreet 6', 'total_price': 72}


In [16]:
import requests

# URL for the register API
url = 'http://localhost:5000/user/register'

# User data to test the registration
user_data = {
    'username': 'new_user',         # Replace with the desired username
    'password': 'new_password',     # Replace with the desired password
    'name': 'Test User',            # Replace with the actual name
    'email': 'testuser@example.com',# Replace with the actual email
    'mobile_number': '1234567890'   # Replace with the actual mobile number
}

# Send POST request to the API
response = requests.post(url, json=user_data)

# Check the response
if response.status_code == 201:
    print("Test Passed: User registered successfully")
    print("Response:", response.json())  # Display the API response
else:
    print(f"Test Failed: Error occurred (status code {response.status_code})")
    print("Response:", response.json())


127.0.0.1 - - [04/Nov/2024 11:04:44] "POST /user/register HTTP/1.1" 201 -


Test Passed: User registered successfully
Response: {'message': 'User registered successfully'}


In [17]:
import requests

def test_get_admin_by_id(admin_id):
    url = f'http://127.0.0.1:5000/admin?admin_id=1'  # Adjust the URL if necessary
    response = requests.get(url)

    if response.status_code == 200:
        print("Admin Details:", response.json())
    else:
        print("Error:", response.json())

# Replace '1' with the actual admin_id you want to test
test_get_admin_by_id(1)

127.0.0.1 - - [04/Nov/2024 11:04:46] "GET /admin?admin_id=1 HTTP/1.1" 200 -


Admin Details: {'_id': '6727cf8de5a70953d4643e27', 'admin_id': 1, 'email': 'admin1@example.com', 'mobile_number': '987-654-3210', 'name': 'Admin One', 'password': 'adminpass1', 'theatre': 'AMC Mainstreet 6', 'username': 'admin1'}
